In [ ]:
from os import getenv
from connect_db import connect_db
con = connect_db(
    username=getenv("ORACLE_USERNAME"),
    password=getenv("ORACLE_PASSWORD"),
    host=getenv("ORACLE_HOST"),
    port=int(getenv("ORACLE_PORT"))
)
cur = con.cursor()

In [ ]:
from dataclasses import dataclass
from datetime import datetime

@dataclass(slots=True)
class CrewInfo:
    can_use_after: datetime
    airport: str
    accumulated_hours: int

stmt = "SELECT EMPLOYEE.EMPLOYEE_ID FROM EMPLOYEE JOIN PILOT ON EMPLOYEE.EMPLOYEE_ID = PILOT.EMPLOYEE_ID"

PILOTS_INFO: dict[str, CrewInfo] = {}
for row in cur.execute(stmt):
    PILOTS_INFO[row[0]] = CrewInfo(datetime.min, "KUL", 0)

In [ ]:
stmt = "SELECT EMPLOYEE.EMPLOYEE_ID FROM EMPLOYEE JOIN FLIGHT_ATTENDANT ON EMPLOYEE.EMPLOYEE_ID = FLIGHT_ATTENDANT.EMPLOYEE_ID"

FA_INFO: dict[str, CrewInfo] = {}
for row in cur.execute(stmt):
    FA_INFO[row[0]] = CrewInfo(datetime.min, "KUL", 0)

In [ ]:
from collections import defaultdict
stmt = """SELECT PILOT.EMPLOYEE_ID, AIRCRAFT_MODEL_ID, FLIGHT_HOUR FROM EMPLOYEE
    JOIN PILOT ON EMPLOYEE.EMPLOYEE_ID = PILOT.EMPLOYEE_ID
    JOIN FLIGHT_HOUR ON PILOT.PILOT_ID = FLIGHT_HOUR.PILOT_ID"""

PILOTS_HOURS: defaultdict[str, dict[str, int]] = defaultdict(dict)

for row in cur.execute(stmt):
    PILOTS_HOURS[row[0]][row[1]] = row[2]

In [ ]:
stmt = """SELECT FLIGHT_ATTENDANT.EMPLOYEE_ID, YEARS_EXPERIENCE FROM EMPLOYEE
    JOIN FLIGHT_ATTENDANT ON EMPLOYEE.EMPLOYEE_ID = FLIGHT_ATTENDANT.EMPLOYEE_ID"""

FA_HOURS: dict[str, int] = {}

for row in cur.execute(stmt):
    FA_HOURS[row[0]] = row[1]

In [ ]:
stmt = """SELECT AIRCRAFT_MODEL.AIRCRAFT_MODEL_ID, NUMBER_OF_SEAT FROM AIRCRAFT
    JOIN AIRCRAFT_MODEL ON AIRCRAFT.AIRCRAFT_MODEL_ID = AIRCRAFT_MODEL.AIRCRAFT_MODEL_ID """
AIRCRAFT_MODEL_SEATS: dict[str, int] = {}
for row in cur.execute(stmt):
    AIRCRAFT_MODEL_SEATS[row[0]] = row[1]

In [ ]:
from typing import Iterable
from datetime import timedelta

ROLES = ("CPTN", "FSOF", "FLEN", "FLEN", "FSOF")
def get_number_of_pilot_roles(number_of_seats: int, duration: float) -> list[str]:
    need_backup = duration > 12
    if number_of_seats > 400:
        n_pilot = 5
    elif number_of_seats > 300:
        n_pilot = 3
    else:
        n_pilot = 2

    out = []
    for _, role in zip(range(n_pilot), ROLES):
        out.append(role)
        if need_backup:
            out.append(role)
    return out

def choose_pilots(aircraft_model_id: str, duration: float, start_time: datetime, start_airport: str) -> Iterable[tuple[str, str]]:
    roles = get_number_of_pilot_roles(AIRCRAFT_MODEL_SEATS[aircraft_model_id], duration)
    number_of_pilots_needed = len(roles)

    most_free_pilots = sorted(filter(lambda t: aircraft_model_id in PILOTS_HOURS[t[0]], PILOTS_INFO.items()), key=lambda t: t[1].can_use_after)
    k = 0
    pilots_chosen = set()

    for pilot, info in most_free_pilots:
        if start_time > info.can_use_after and start_airport == info.airport:
            pilots_chosen.add(pilot)
            k += 1
            if k == number_of_pilots_needed:
                break

    if k < number_of_pilots_needed:
        for pilot, info in most_free_pilots:
            if pilot not in pilots_chosen:
                if start_time > info.can_use_after:
                    pilots_chosen.add(pilot)
                    k += 1
                    if k == number_of_pilots_needed:
                        break

    pilots_chosen = sorted(pilots_chosen, key=lambda id: PILOTS_HOURS[id][aircraft_model_id], reverse=True)
    for pilot_id, role in zip(pilots_chosen, roles):
        yield pilot_id, role

from typing import NamedTuple
class Rest(NamedTuple):
    rest_time: datetime
    duration: float

REST_LOG: defaultdict[str, list[Rest]] = defaultdict(list)
def use_pilot(pilot_id: str, duration: float, end_time: datetime, end_airport: str):
    pilot_info = PILOTS_INFO[pilot_id]
    pilot_info.airport = end_airport
    pilot_info.accumulated_hours += duration
    pilot_info.can_use_after += timedelta(hours=duration)

    if pilot_info.accumulated_hours > 8:
        rest_time = max(12, pilot_info.accumulated_hours)
        pilot_info.accumulated_hours = 0
        REST_LOG[pilot_id].append(Rest(end_time + timedelta(minutes=30), rest_time))
        pilot_info.can_use_after += timedelta(hours=rest_time, minutes=30)

In [ ]:
# FA_ROLES = ("PURS", "FLAT")
def get_number_of_flight_attendants_roles(number_of_seats: int, duration: float) -> list[str]:
    need_backup = duration > 12
    number_of_fa = number_of_seats // 50
    out = []
    for _ in range(number_of_fa):
        out.append("PURS")
        if need_backup:
            out.append("PURS")
    for _ in range(number_of_fa - 1):
        out.append("FLAT")
        if need_backup:
            out.append("FLAT")
    return out

def choose_fa(aircraft_model_id: str, duration: float, start_time: datetime, start_airport: str) -> Iterable[tuple[str, str]]:
    roles = get_number_of_flight_attendants_roles(AIRCRAFT_MODEL_SEATS[aircraft_model_id], duration)
    number_of_fa_needed = len(roles)

    sorted_fa = sorted(FA_INFO.items(), key=lambda t: t[1].can_use_after)
    k = 0
    fa_chosen = set()

    for fa, info in sorted_fa:
        if start_time > info.can_use_after and start_airport == info.airport:
            fa_chosen.add(fa)
            k += 1
            if k == number_of_fa_needed:
                break

    if k < number_of_fa_needed:
        for fa, info in sorted_fa:
            if fa not in fa_chosen:
                if start_time > info.can_use_after:
                    fa_chosen.add(fa)
                    k += 1
                    if k == number_of_fa_needed:
                        break

    fa_chosen = sorted(fa_chosen, key=lambda id: FA_HOURS[id], reverse=True)
    for fa_id, role in zip(fa_chosen, roles):
        yield fa_id, role


def use_fa(fa_id: str, duration: float, end_airport: str):
    fa_info = FA_INFO[fa_id]
    fa_info.airport = end_airport
    fa_info.accumulated_hours += duration
    fa_info.can_use_after += timedelta(hours=duration)

In [ ]:
import numpy as np
def deviate_time(dt: datetime, minutes: int) -> datetime:
    minutes_range = np.arange(minutes)
    weights = np.concatenate((
        np.ones(int(np.round(minutes / 3)), dtype=int) * 0.8 / int(np.round(minutes / 3)),
        np.ones(int(np.round(minutes / 3)), dtype=int) * 0.15 / int(np.round(minutes / 3)),
        np.ones(int(np.round(minutes / 3)), dtype=int) * 0.05 / int(np.round(minutes / 3))
    ))
    return dt - timedelta(minutes=np.random.choice(minutes_range, size=1, p=weights).item())

In [ ]:
starting_date = datetime(2023, 1, 1)
now = datetime(2024, 4, 19, 15)

stmt = """SELECT FLIGHT_ID, AIRCRAFT_MODEL_ID, EST_DURATION_IN_HOUR, DEPARTURE_DATETIME, FROM_AIRPORT_ID, TO_AIRPORT_ID
FROM FLIGHT
    JOIN AIRCRAFT ON FLIGHT.AIRCRAFT_ID = AIRCRAFT.AIRCRAFT_ID
    JOIN ROUTE ON FLIGHT.ROUTE_ID = ROUTE.ROUTE_ID"""
sql = "    INTO FLIGHT_CREW_SCHEDULE (EMPLOYEE_ID, FLIGHT_ID, CREW_ROLE_ID, CLOCK_IN_TIME, CLOCK_OUT_TIME) VALUES ({}, {}, '{}', TO_TIMESTAMP('{:%Y-%m-%d %H:%M:%S}', 'YYYY-MM-DD HH24:MI:SS'), TO_TIMESTAMP('{:%Y-%m-%d %H:%M:%S}', 'YYYY-MM-DD HH24:MI:SS'))"

print("INSERT ALL")
k = 0
for row in cur.execute(stmt):
    flight_id = row[0]
    model_id = row[1]
    duration = row[2]
    departure_time = row[3]
    arrival_time = departure_time + timedelta(hours=duration)
    start_airport = row[4]
    end_airport = row[5]
    for emp_id, role in choose_pilots(model_id, duration, departure_time, start_airport):
        print(sql.format(emp_id, flight_id, role, deviate_time(departure_time, 9), deviate_time(arrival_time, 30)))
        use_pilot(emp_id, duration, arrival_time, end_airport)
        k += 1

    for emp_id, role in choose_fa(model_id, duration, departure_time, start_airport):
        print(sql.format(emp_id, flight_id, role, deviate_time(departure_time, 9), deviate_time(arrival_time, 30)))
        use_fa(emp_id, duration, end_airport)
        k += 1

    if k > 2000:
        print("SELECT 1 FROM DUAL;")
        print("INSERT ALL")
        k = 0
print("SELECT 1 FROM DUAL;")

In [ ]:
stmt = """SELECT EMPLOYEE.EMPLOYEE_ID, PILOT_ID FROM EMPLOYEE
    JOIN PILOT ON EMPLOYEE.EMPLOYEE_ID = PILOT.EMPLOYEE_ID"""
EMP_TO_PILOT_ID: dict[str, str] = {row[0]: row[1] for row in cur.execute(stmt)}

In [ ]:
sql = "    INTO REST_LOG (PILOT_ID, START_DATETIME, END_DATETIME) VALUES ('{}', TO_TIMESTAMP('{:%Y-%m-%d %H:%M:%S}', 'YYYY-MM-DD HH24:MI:SS'), TO_TIMESTAMP('{:%Y-%m-%d %H:%M:%S}', 'YYYY-MM-DD HH24:MI:SS'))"

print("INSERT ALL")
k = 0
for emp_id, rest in REST_LOG.items():
    for rest_log in rest:
        print(sql.format(EMP_TO_PILOT_ID[emp_id], rest_log.rest_time, rest_log.rest_time + timedelta(hours=rest_log.duration)))

        k += 1
        if k >= 2000:
            print("SELECT 1 FROM DUAL;")
            print("INSERT ALL")
            k = 0
print("SELECT 1 FROM DUAL;")

In [ ]:
con.close()